In [36]:
#cose da eseguire
#in un terminale (1) -> docker-compose up
#in un altro terminale (2) -> docker exec -it ollama bash
#nel terminale 2 -> ollama pull phi3


In [ ]:
!pip install elasticsearch tqdm openai

In [5]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [6]:
import json

#create the docs_raw variable from the .json file
with open('documents.json','rt') as f_in:
    docs_raw = json.load(f_in)

documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)
        

In [7]:
def build_prompt(query,search_results):
    prompt_template="""
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    If the CONTEXT doesn't cointain the answer, output NONE.

    QUESTION: {question}

    CONTEXT:
    {context}
    """

    
    context = ""

    for doc in search_results:
        context= context + f"section: {doc['section']}\n\nquestion: {doc['question']}\n\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query,context=context)
    return prompt

In [8]:
def llm(prompt):

    response=client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content":prompt}]
    )
    return response.choices[0].message.content
    #import ollama
    #response = ollama.generate(model="gemma:2b", prompt=prompt)
    #return response

In [46]:
print(response['response'])

Yes, you can still enroll in the course after the start date. You will be able to submit homeworks and continue working on the final project at your own pace after it finishes.


In [12]:
query = "the course has already started, can I still enroll?"


In [10]:
def rag(query):
    results = search_elastic(query)
    prompt = build_prompt(query,results)
    response = llm(prompt)
    return response['response']

In [15]:
answer = rag(query)
print(answer)

InternalServerError: Error code: 500 - {'error': {'message': 'model requires more system memory (5.6 GiB) than is available (4.2 GiB)', 'type': 'api_error', 'param': None, 'code': None}}

#elastic search

In [4]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200')
index_name = "course-questions"
import tqdm
#dont run this cell several times, or (idk why) some doc duplicated. If so, try to delete the index ->
#es_client.indices.delete(index=index_name)
from tqdm.auto import tqdm
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)



  0%|          | 0/948 [00:00<?, ?it/s]

In [ ]:
#CREATING INDICES
#they are saved permanently, so you have to run it only 1 time, no need to rerun after power-up

index_settings = {
        "settings": {
            "number_of_shards": 1,
            "number_of_replicas": 0
        },
        "mappings": {
            "properties": {
                "text": {"type": "text"},
                "section": {"type": "text"},
                "question": {"type": "text"},
                "course": {"type": "keyword"} 
            }
        }
    }
    

    
es_client.indices.create(index=index_name, body=index_settings)

In [14]:
#"filter" chooose the document that match with the "course"
#"question^3" means that the field is 3 times more important than the other
#"size" is the number of document retrieve


def search_elastic(query):
    search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^3", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    results_doc = []
    
    for hit in response['hits']['hits']:
        results_doc.append(hit['_source'])
    
    return results_doc

In [2]:
!unistall

/bin/bash: unistall: command not found
